## Indexing

### Grid Layout

The 2D grid uses column-major ordering with shape `(Nx_padded, Ny_padded)`.

- Axis 0 = X (West → East)
- Axis 1 = Y (South → North)
- Access: `field[x, y]`

### Index Mask

Maps grid positions to solution vector indices. Shape: `(Nx_padded, Ny_padded)`.

mask\_index\_inner\_local for `Nx_inner=3`, `Ny_inner=2`

$$
\begin{pmatrix}
-1 & -1 & -1 & -1 \\
-1 & 0 & 3 & -1 \\
-1 & 1 & 4 & -1 \\
-1 & 2 & 5 & -1 \\
-1 & -1 & -1 & -1
\end{pmatrix}
$$

where rows = x-index, columns = y-index, and $-1$ = boundary (ghost cell).